<img src='otus.png'>

# Analyzing text data

# Thematic modeling

A topic is a semantically homogeneous cluster of texts characterized by specialized terminology.

The thematic model automatically determines which topics each document from the collection of documents relates to, as well as which words (terms) characterize each topic.

<img src="https://upload.wikimedia.org/wikipedia/commons/d/d5/%D0%A2%D0%B5%D0%BC%D0%B0%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B0%D1%8F_%D0%BC%D0%BE%D0%B4%D0%B5%D0%BB%D1%8C.png">

## Topic modeling hypotheses

- the word order in the document is not important
- the order of documents in the collection is not important
- each term w in the document is associated with some topic
- hypothesis of conditional independence:
$$p(w|d,t) = p(w|t)$$

In [1]:
# Author: Olivier Grisel <olivier.grisel@ensta.org>
#         Lars Buitinck
#         Chyi-Kwei Yau <chyikwei.yau@gmail.com>
# License: BSD 3 clause

from __future__ import print_function
from time import time

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.datasets import fetch_20newsgroups

n_samples = 2000
n_features = 1000
n_components = 10
n_top_words = 20


def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()


# Load the 20 newsgroups dataset and vectorize it. We use a few heuristics
# to filter out useless terms early on: the posts are stripped of headers,
# footers and quoted replies, and common English words, words occurring in
# only one document or in at least 95% of the documents are removed.

print("Loading dataset...")
t0 = time()
dataset = fetch_20newsgroups(shuffle=True, random_state=1,
                             remove=('headers', 'footers', 'quotes'))
data_samples = dataset.data[:n_samples]
print("done in %0.3fs." % (time() - t0))

Loading dataset...
done in 13.066s.


In [2]:
len(data_samples)

2000

In [3]:
data_samples[0]

"Well i'm not sure about the story nad it did seem biased. What\nI disagree with is your statement that the U.S. Media is out to\nruin Israels reputation. That is rediculous. The U.S. media is\nthe most pro-israeli media in the world. Having lived in Europe\nI realize that incidences such as the one described in the\nletter have occured. The U.S. media as a whole seem to try to\nignore them. The U.S. is subsidizing Israels existance and the\nEuropeans are not (at least not to the same degree). So I think\nthat might be a reason they report more clearly on the\natrocities.\n\tWhat is a shame is that in Austria, daily reports of\nthe inhuman acts commited by Israeli soldiers and the blessing\nreceived from the Government makes some of the Holocaust guilt\ngo away. After all, look how the Jews are treating other races\nwhen they got power. It is unfortunate.\n"

In [4]:
# Use tf-idf features for NMF.
print("Extracting tf-idf features for NMF...")
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2,
                                   max_features=n_features,
                                   stop_words='english')
t0 = time()
tfidf = tfidf_vectorizer.fit_transform(data_samples)
print("done in %0.3fs." % (time() - t0))

# Use tf (raw term count) features for LDA.
print("Extracting tf features for LDA...")
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2,
                                max_features=n_features,
                                stop_words='english')
t0 = time()
tf = tf_vectorizer.fit_transform(data_samples)
print("done in %0.3fs." % (time() - t0))
print()

# Fit the NMF model
print("Fitting the NMF model (Frobenius norm) with tf-idf features, "
      "n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
t0 = time()
nmf = NMF(n_components=n_components, random_state=1,
          alpha=.1, l1_ratio=.5).fit(tfidf)
print("done in %0.3fs." % (time() - t0))

print("\nTopics in NMF model (Frobenius norm):")
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(nmf, tfidf_feature_names, n_top_words)

# Fit the NMF model
print("Fitting the NMF model (generalized Kullback-Leibler divergence) with "
      "tf-idf features, n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
t0 = time()
nmf = NMF(n_components=n_components, random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5).fit(tfidf)
print("done in %0.3fs." % (time() - t0))

print("\nTopics in NMF model (generalized Kullback-Leibler divergence):")
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(nmf, tfidf_feature_names, n_top_words)

print("Fitting LDA models with tf features, "
      "n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
lda = LatentDirichletAllocation(n_components=n_components, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)
t0 = time()
lda.fit(tf)
print("done in %0.3fs." % (time() - t0))

print("\nTopics in LDA model:")
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)

Extracting tf-idf features for NMF...
done in 0.799s.
Extracting tf features for LDA...
done in 1.060s.

Fitting the NMF model (Frobenius norm) with tf-idf features, n_samples=2000 and n_features=1000...
done in 1.496s.

Topics in NMF model (Frobenius norm):
Topic #0: just people don think like know time good make way really say right ve want did ll new use years
Topic #1: windows use dos using window program os drivers application help software pc running ms screen files version card code work
Topic #2: god jesus bible faith christian christ christians does heaven sin believe lord life church mary atheism belief human love religion
Topic #3: thanks know does mail advance hi info interested email anybody looking card help like appreciated information send list video need
Topic #4: car cars tires miles 00 new engine insurance price condition oil power speed good 000 brake year models used bought
Topic #5: edu soon com send university internet mit ftp mail cc pub article information hope

In [5]:
lda.components_

array([[ 4.96604155,  4.3537397 , 21.42539886, ...,  1.57926488,
         1.33933502,  1.20988436],
       [ 0.48391034,  1.85845783, 14.04720958, ..., 74.59501615,
        59.36116266,  0.27698642],
       [ 0.18708486,  0.13728929,  0.31409364, ...,  1.02679042,
         2.56259123,  0.13662652],
       ...,
       [ 3.22343848, 39.1368944 , 11.24910558, ..., 23.37779481,
         3.06315114,  0.15230766],
       [ 1.41871388, 47.53082031, 16.14390001, ..., 82.46751192,
        16.51319941, 28.11660323],
       [ 4.02759659,  1.24781464, 13.26101699, ..., 29.0225105 ,
         0.24834416,  0.13033208]])

In [6]:
lda.components_.shape

(10, 1000)

# Naive Bayes
* We know the label of each document
* Each document has only one label


What can you do if there is no tag information?
#### Clustering problem
You can solve using the EM-algorithm:
* E-step - calculate expectations of which document relates to which topic
* M-step - using Naive Bayes to determine the probabilities $ p (w | t) $ with fixed labels


## EM algorithm

Solves the clustering problem.
Selects some model parameters for data in which the answer is unknown.

Expectation step:
* fix model parameters
* count the values ​​of hidden variables
Maximization step:
* commit hidden variables
* calculate model parameters

Repeat until convergence.

There is a mathematical justification for the fact that the method converges to a local extremum, at each step the value of the likelihood function does not decrease (the likelihood $ p (\ theta | \ mathcal {X}) $ - how plausible is the model for the given parameters, how well does it describe the data)

A special case of the EM algorithm is ** k-means **.
Cluster labels - latent variables
Model Parameters - Cluster Centers

<img src = "kmeans.png">

Another variant of the EM algorithm is the separation of a Gaussian Mixture Model (GMM)

Model parameters - cluster center and covariance matrix (here describes the shape of the Mogomeric normal distribution, or Gaussian)
Hidden variables - the probability of belonging to each Gaussian (the cluster label is chosen as the most likely cluster)


<img src="gauss.png">

## PLSA

What if each document can have many tags?

Consider the model:
* Each word in document $ d $ is generated from some topic $ t \ in T $
* Document generated by some distribution over topics $ p (t | d) $
* Word generated from topic (not from document) $ p (w | d, t) = p (w | d) $
* We get the likelihood: $$ p (w | d) = \ sum_ {t \ in T} p (w | t) p (t | d) $$

The resulting model - probabilistic latent semantic analysis, pLSA, probabilistic latent semantic analysis

http://www.machinelearning.ru/wiki/index.php?title=%D0%92%D0%B5%D1%80%D0%BE%D1%8F%D1%82%D0%BD%D0%BE%D1%81%D1%82%D0%BD%D1%8B%D0%B9_%D0%BB%D0%B0%D1%82%D0%B5%D0%BD%D1%82%D0%BD%D1%8B%D0%B9_%D1%81%D0%B5%D0%BC%D0%B0%D0%BD%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B8%D0%B9_%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7

Training:
We need values:
* $ p (w | t) $ - probabilities of words in topics, denote $ \ phi_ {wt} $

* $ p (t | d) $ - probabilities of topics in documents, denote $ \ theta_ {td} $

E-step:
* fix $ \ phi_ {wt} $ and $ \ theta_ {td} $
* calculate $$ p (t | d, w) = \ frac {\ phi_ {wt} \ theta_ {td}} {\ sum_ {s \ in T} \ phi_ {ws} \ theta_ {sd}} $$ for all topics, for each document, for each term
* calculate the number of terms that is generated in the document $ d $ by the topic $ t $ $$ n_ {dwt} = n_ {dw} p (t | d, w) $$

M-step:
* for the calculated $ p (t | d, w) $ update the approximations of the $ \ phi_ {wt} $ and $ \ theta_ {td} $ model
* $$ n_ {wt} = \ sum_d n_ {dwt} $$ $$ n_ {td} = \ sum_ {w \ in d} n_ {dwt} $$ $$ n_t = \ sum_w n_ {wt} $$
* $$ \ theta_ {td} = \ frac {n_ {td}} {n_d} $$ $$ \ phi_ {wt} = \ frac {n_ {wt}} {n_t} $$


You can not store the matrix $ n_ {dwt} $, but iterate over the documents and sum up $ n_ {wt} $ and $ n_ {td} $
* Many local extrema
* Many parameters, the model is retrained
* It is necessary to achieve not a local minimum, but to achieve interpretability - to find a "good" minimum

## LDA

In general, to improve pLSA, regularization is added to the log likelihood:

$$\sum_{d \in D} \sum_{w \in d} n_{dw} ln \sum_{t \in T} \phi_{wt} \theta_{td} + \sum_i \tau_i R_i(\Phi, \Theta)$$

If we add the prior distribution - the Dirichlet distribution, we get the LDA - Latent Dirichlet Allocation algorithm

As a result, we get a "good" interpreted solution (better than with pLSA)

One document can contain several topics.
We compose a hierarchical model:
* the first level is a mixture, the components of which are responsible for the themes
* the second level is a multinomial variable with a priori Dirichlet distribution, which determines the "distribution over topics" in the document

Training:
* Gibs sampling
* online variational bayes

## Scheme for solving NLP problems:
1. Data collection:
     - API
     - site crawling
     - known datasets
     - Mechanical Turk, Toloka, etc.

2. Choice of quality metrics.

3. Data preprocessing.

4. Vectorization of data.
     - Bag Of Words
     - TF-IDF
     - Word-Embeddings (Word2Vec, Glove, Fasttext)
     - Text embeddings (ELMO, Infersent)

5. Model selection.
     - GridSerch
     - RandomSearch

First of all, it is worth looking at linear models and neural networks.

6. Evaluation of the solution in production.

7. GoTo 1.

### Additional materials

http://www.machinelearning.ru/wiki/images/8/82/BMMO11_14.pdf  
http://www.machinelearning.ru/wiki/images/f/f7/DirichletProcessNotes.pdf 

LDA material:
- http://www.cs.columbia.edu/~blei/talks/Blei_Topic_Modeling_Workshop_2013.pdf
- http://www.cs.columbia.edu/~blei/papers/Blei2012.pdf
- http://www.machinelearning.ru/wiki/images/8/82/BMMO11_14.pdf
- http://www.machinelearning.ru/wiki/images/f/f7/DirichletProcessNotes.pdf 

coherence article:
- http://www.aclweb.org/anthology/N10-1012
